### This file is for data manipulation of different visualisaton requirement for VGI_Challenge 3. 

###### Note: If a dataframe has _NA at the end, it means it only contains completed trips, if not, it has all the trip data

In [1]:
import pandas as pd  # type: ignore
from pathlib import Path

##### Join two datasets and have pickup and drop off details with lat/lon

In [3]:
table1 = pd.read_excel(Path("./dataset/FLEXI_bus_stops.xls"))
table2 = pd.read_excel(Path("./dataset/FLEXI_trip_data.xls"))

merged_data = table2.merge(table1, left_on="Pickup ID", right_on="index") \
                    .merge(table1, left_on="Dropoff ID", right_on="index", suffixes=('', '_dropoff')) \
                    .rename(columns={
                        "index": "pickup_index",
                        "name": "pickup_name",
                        "district": "pickup_district",
                        "latitude": "pickup_latitude",
                        "longitude": "pickup_longitude"
                    }) \
                    .drop(columns=["Pickup ID", "Dropoff ID"])

merged_data['Actual Pickup Time'] = pd.to_datetime(merged_data['Actual Pickup Time'])
merged_data['Actual Dropoff Time'] = pd.to_datetime(merged_data['Actual Dropoff Time'])
merged_data['Pickup Hour'] = merged_data['Actual Pickup Time'].dt.hour
merged_data['Dropoff Hour'] = merged_data['Actual Dropoff Time'].dt.hour
merged_data['Pickup Day'] = merged_data['Actual Pickup Time'].dt.dayofweek  # 0 = Monday, 1= Tuesday, 2 = Wednesday ... 


merged_data = merged_data[merged_data['Passenger status'] != 'Cancelled']
merged_data

,Booking ID,Status,Passenger status,Passengers,Actual Pickup Time,Actual Dropoff Time,pickup_index,pickup_name,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Pickup Hour,Dropoff Hour,Pickup Day
3,253782,Validated,Trip completed,2,2024-09-01 09:14:00,2024-09-01 09:25:00,40,"Irfersdorf, Am Kirchplatz",Irfersdorf,48.994215,11.461103,21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,9,9,6
5,251869,Validated,Trip completed,3,2024-09-01 09:54:00,2024-09-01 10:06:00,27,"Beilngries, Hafen",Beilngries,49.041112,11.471240,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,9,10,6
6,251083,Validated,Trip completed,1,2024-09-01 10:39:00,2024-09-01 10:46:00,20,"Beilngries, ZOB",Beilngries,49.032173,11.476295,9,"Unteremmendorf, Wendeschleife",Unteremmendorf,48.992112,11.434157,10,10,6
9,253916,Validated,Trip completed,2,2024-09-01 10:36:00,2024-09-01 10:52:00,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,10,10,6
10,251100,Validated,Trip completed,1,2024-09-01 11:23:00,2024-09-01 11:34:00,50,"Wolfsbuch, Fuchsengasse",Wolfsbuch,48.976827,11.559612,38,"Paulushofen, Dorfstra√üe",Paulushofen,49.010337,11.502283,11,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,270320,Validated,Trip completed,1,2024-09-30 19:15:00,2024-09-30 19:20:00,30,"Beilngries, Frauenkirche",Beilngries,49.036378,11.470632,10,"Pfraundorf, Dorfplatz",Pfraundorf,49.005142,11.445253,19,19,0
3425,263021,Validated,Trip completed,1,2024-09-30 20:08:00,2024-09-30 20:14:00,30,"Beilngries, Frauenkirche",Beilngries,49.036378,11.470632,40,"Irfersdorf, Am Kirchplatz",Irfersdorf,48.994215,11.461103,20,20,0
3427,269620,Validated,Trip completed,1,2024-09-30 20:55:00,2024-09-30 21:06:00,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,29,"Beilngries, Rathaus",Beilngries,49.035103,11.474083,20,21,0
3430,268219,Validated,Trip completed,1,2024-09-30 21:20:00,2024-09-30 21:23:00,4,"Kinding, Marktplatz",Kinding,49.000562,11.382790,1,"Enkering, Maibaum",Enkering,48.992498,11.363953,21,21,0


#### Most demanded routes / origin-destination pair

In [5]:
origin_destination_pair = merged_data.groupby(['pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 'index_dropoff', \
                                               'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                                            .size() \
                                            .reset_index(name="Frequency") \
                                            .sort_values(by='Frequency', ascending=False)


origin_destination_pair 

,pickup_index,pickup_name,Passengers,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Frequency
295,28,"Beilngries, Neumarkter Stra√üe",1,Beilngries,49.036912,11.471075,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,39
373,31,"Beilngries, Eichstaetter Strasse / Tankstelle",1,Beilngries,49.035227,11.467885,15,"Hirschberg, Dorfkapelle",Hirschberg,49.038150,11.451488,38
205,19,"Beilngries, Ringstrasse",1,Beilngries,49.033832,11.471982,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,37
19,0,"Kinding, Bahnhof",1,Kinding,48.992168,11.377365,28,"Beilngries, Neumarkter Stra√üe",Beilngries,49.036912,11.471075,37
12,0,"Kinding, Bahnhof",1,Kinding,48.992168,11.377365,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,31
...,...,...,...,...,...,...,...,...,...,...,...,...
262,21,"Beilngries, Deutscher Hof",4,Beilngries,49.033525,11.475793,1,"Enkering, Maibaum",Enkering,48.992498,11.363953,1
261,21,"Beilngries, Deutscher Hof",3,Beilngries,49.033525,11.475793,65,"Litterzhofen, Kirche",Plakstetten,49.069082,11.407390,1
260,21,"Beilngries, Deutscher Hof",3,Beilngries,49.033525,11.475793,40,"Irfersdorf, Am Kirchplatz",Irfersdorf,48.994215,11.461103,1
259,21,"Beilngries, Deutscher Hof",3,Beilngries,49.033525,11.475793,38,"Paulushofen, Dorfstra√üe",Paulushofen,49.010337,11.502283,1


#### Common travel patterns w.r.t time of the day 

| Time      | German      | English     |
|---------------|---------------|---------------|
| 06:00–10:00 | Morgen | morning  |
| 10:00–12:00  | Vormittag  | late morning  |
| 12:00–13:00  | Mittag  | noon  |
| 13:00–17:00  | NachMittag  | afternoon  |
| 17:00–0:00  | Abend  | evening  |
| 0:00–06:00  | Nacht  | night  |

In [52]:
def categorize_time_of_day(hour):
     if 6 <= hour < 10:
            return 'Morgen (morning)'
     elif 10 <= hour < 12:
            return 'Vormittag (late morning)'
     elif 12 <= hour < 13:
            return 'Mittag (noon)'
     elif 13 <= hour < 17:
           return 'Nachmittag (afternoon)'
     elif 17 <= hour < 24:
           return 'Abend (evening)'
     else:
        return 'Nacht (night)'

In [53]:

travel_pattern = merged_data.copy()
travel_pattern['time_of_day'] = merged_data['Actual Pickup Time'].dt.hour.apply(categorize_time_of_day)

travel_pattern_NA = travel_pattern[travel_pattern['Passenger status'] != 'Cancelled']  #_NA includes only completed trips

travel_pattern_NA = travel_pattern_NA.groupby(['time_of_day', 'pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 'index_dropoff', \
                                               'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                                            .size() \
                                            .reset_index(name="Frequency") \
                                            .sort_values(by='Frequency', ascending=False) 

travel_pattern = travel_pattern.groupby(['time_of_day', 'pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 'index_dropoff', \
                                               'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                                            .size() \
                                            .reset_index(name="Frequency") \
                                            .sort_values(by='Frequency', ascending=False) 

travel_pattern

,time_of_day,pickup_index,pickup_name,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Frequency
10,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,52
16,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,28,"Beilngries, Neumarkter Stra√üe",Beilngries,49.036912,11.471075,46
650,Nachmittag (afternoon),19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,38
557,Nachmittag (afternoon),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,33
508,Morgen (morning),44,"Aschbuch, Altenzeller Weg",Aschbuch,48.982160,11.494917,26,"Beilngries, Kanalsiedlung",Beilngries,49.044913,11.469383,32
...,...,...,...,...,...,...,...,...,...,...,...,...
430,Morgen (morning),15,"Hirschberg, Dorfkapelle",Hirschberg,49.038150,11.451488,29,"Beilngries, Rathaus",Beilngries,49.035103,11.474083,1
431,Morgen (morning),15,"Hirschberg, Dorfkapelle",Hirschberg,49.038150,11.451488,30,"Beilngries, Frauenkirche",Beilngries,49.036378,11.470632,1
433,Morgen (morning),15,"Hirschberg, Dorfkapelle",Hirschberg,49.038150,11.451488,36,"Paulushofen, Am Haar",Paulushofen,49.013625,11.502735,1
435,Morgen (morning),16,"Hirschberg, Fuerstenstrasse",Hirschberg,49.036600,11.454518,28,"Beilngries, Neumarkter Stra√üe",Beilngries,49.036912,11.471075,1


#### Average duration of these trips

In [54]:
def calc_time_difference(fromdate, todate):
    return (fromdate - todate).dt.total_seconds() / 60

In [55]:
average_duration = merged_data.copy()
average_duration['Trip duration (minutes)'] = calc_time_difference(average_duration['Actual Dropoff Time'], average_duration['Actual Pickup Time'])
average_duration['time_of_day'] = merged_data['Actual Pickup Time'].dt.hour.apply(categorize_time_of_day)

average_duration_NA = average_duration[average_duration['Passenger status'] != 'Cancelled']

average_duration =  average_duration.groupby(['time_of_day', 'pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 
                                    'index_dropoff', 'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                          .agg(Frequency=('Trip duration (minutes)', 'size'), 
                               Avg_Duration=('Trip duration (minutes)', 'mean')) \
                          .reset_index()

average_duration_NA =  average_duration_NA.groupby(['time_of_day', 'pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 
                                    'index_dropoff', 'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                          .agg(Frequency=('Trip duration (minutes)', 'size'), 
                               Avg_Duration=('Trip duration (minutes)', 'mean')) \
                          .reset_index()

average_duration_NA

,time_of_day,pickup_index,pickup_name,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Frequency,Avg_Duration
0,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,3,"Kinding, Beilngrieser Stra√üe",Kinding,49.002445,11.390002,2,12.000000
1,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,6,"Haunstetten, Kirche",Haunstetten,49.015158,11.408473,3,5.666667
2,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,9,"Unteremmendorf, Wendeschleife",Unteremmendorf,48.992112,11.434157,2,8.500000
3,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,10,"Pfraundorf, Dorfplatz",Pfraundorf,49.005142,11.445253,2,7.000000
4,Abend (evening),0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,11,"Kratzmuehle, Seerestaurant",Pfraundorf,49.004607,11.447705,1,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,Vormittag (late morning),64,"Plankstetten, Biberbacher Stra√üe",Biberbach,49.068532,11.455142,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,1,19.000000
760,Vormittag (late morning),64,"Plankstetten, Biberbacher Stra√üe",Biberbach,49.068532,11.455142,31,"Beilngries, Eichstaetter Strasse / Tankstelle",Beilngries,49.035227,11.467885,1,6.000000
761,Vormittag (late morning),67,Kaldorf,Wiesenhofen,49.048472,11.431715,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,1,6.000000
762,Vormittag (late morning),67,Kaldorf,Wiesenhofen,49.048472,11.431715,25,"Beilngries, Ottmaringer Stra√üe",Beilngries,49.038975,11.480128,2,10.000000


#### Average Passenger count in each of these trips

In [56]:
average_passenger = merged_data.copy()
average_passenger['Trip duration (minutes)'] = calc_time_difference(average_passenger['Actual Dropoff Time'], average_passenger['Actual Pickup Time'])
average_passenger['time_of_day'] = merged_data['Actual Pickup Time'].dt.hour.apply(categorize_time_of_day)

average_passenger_NA = average_passenger[average_passenger['Passenger status'] != 'Cancelled']

average_passenger =  average_passenger.groupby(['time_of_day', 'pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 
                                    'index_dropoff', 'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                          .agg(Frequency=('Trip duration (minutes)', 'size'), 
                               Average_Passenger=('Passengers', 'mean')) \
                          .reset_index()

average_passenger_NA =  average_passenger_NA.groupby(['time_of_day', 'pickup_index', 'pickup_name', 'pickup_district', 'pickup_latitude', 'pickup_longitude', 
                                    'index_dropoff', 'name_dropoff', 'district_dropoff', 'latitude_dropoff', 'longitude_dropoff']) \
                          .agg(Frequency=('Trip duration (minutes)', 'size'), 
                               Average_Passenger=('Passengers', 'mean')) \
                          .reset_index() \
                          .sort_values(by='Average_Passenger', ascending=False)

average_passenger_NA

,time_of_day,pickup_index,pickup_name,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Frequency,Average_Passenger
382,Morgen (morning),49,"Wolfsbuch, Am Schacher",Wolfsbuch,48.974490,11.562987,21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,1,7.0
260,Mittag (noon),49,"Wolfsbuch, Am Schacher",Wolfsbuch,48.974490,11.562987,29,"Beilngries, Rathaus",Beilngries,49.035103,11.474083,1,7.0
395,Morgen (morning),59,"Kevenhuell, Kirche",Leising,49.058008,11.516585,21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,2,6.0
86,Abend (evening),21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,45,"Aschbuch, Lindenstra√üe",Aschbuch,48.978988,11.491803,1,5.0
112,Abend (evening),30,"Beilngries, Frauenkirche",Beilngries,49.036378,11.470632,1,"Enkering, Maibaum",Enkering,48.992498,11.363953,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Morgen (morning),2,"Enkering, Feuerwehrhaus",Enkering,48.993652,11.361228,30,"Beilngries, Frauenkirche",Beilngries,49.036378,11.470632,1,1.0
288,Morgen (morning),2,"Enkering, Feuerwehrhaus",Enkering,48.993652,11.361228,34,"Beilngries, Volksfestplatz",Beilngries,49.031120,11.469645,2,1.0
289,Morgen (morning),3,"Kinding, Beilngrieser Stra√üe",Kinding,49.002445,11.390002,9,"Unteremmendorf, Wendeschleife",Unteremmendorf,48.992112,11.434157,2,1.0
290,Morgen (morning),3,"Kinding, Beilngrieser Stra√üe",Kinding,49.002445,11.390002,10,"Pfraundorf, Dorfplatz",Pfraundorf,49.005142,11.445253,1,1.0


#### Average Revenue per trip

In [57]:
average_revenue = average_passenger_NA.copy()

average_revenue = average_revenue.assign(Average_Revenue=average_revenue['Average_Passenger'] * 2.0) \
                                                    .sort_values(by='Average_Revenue', ascending=False)

average_revenue


,time_of_day,pickup_index,pickup_name,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Frequency,Average_Passenger,Average_Revenue
382,Morgen (morning),49,"Wolfsbuch, Am Schacher",Wolfsbuch,48.974490,11.562987,21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,1,7.0,14.0
260,Mittag (noon),49,"Wolfsbuch, Am Schacher",Wolfsbuch,48.974490,11.562987,29,"Beilngries, Rathaus",Beilngries,49.035103,11.474083,1,7.0,14.0
395,Morgen (morning),59,"Kevenhuell, Kirche",Leising,49.058008,11.516585,21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,2,6.0,12.0
86,Abend (evening),21,"Beilngries, Deutscher Hof",Beilngries,49.033525,11.475793,45,"Aschbuch, Lindenstra√üe",Aschbuch,48.978988,11.491803,1,5.0,10.0
112,Abend (evening),30,"Beilngries, Frauenkirche",Beilngries,49.036378,11.470632,1,"Enkering, Maibaum",Enkering,48.992498,11.363953,1,5.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,Nachmittag (afternoon),39,"Neuzell, Feuerwehr",Neuzell,48.997518,11.482948,45,"Aschbuch, Lindenstra√üe",Aschbuch,48.978988,11.491803,1,1.0,2.0
598,Nachmittag (afternoon),46,"Aschbuch, Waldsiedlung",Aschbuch,48.976207,11.491877,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,1,1.0,2.0
597,Nachmittag (afternoon),43,"Grampersdorf, Gewerbegebiet",Grampersdorf,48.976818,11.478615,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,1,1.0,2.0
596,Nachmittag (afternoon),42,"Grampersdorf, Landstrasse",Grampersdorf,48.970778,11.476563,48,Arnbuch,Arnbuch,48.975027,11.534193,4,1.0,2.0
